In [26]:
import tensorflow as tf
import tensorflow as tf

# Define the F1 score metric
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision_result = self.precision.result()
        recall_result = self.recall.result()
        return 2 * ((precision_result * recall_result) / (precision_result + recall_result + tf.keras.backend.epsilon()))

    def reset_state(self):
        self.precision.reset_state()
        self.recall.reset_state()
        
import pickle5
from sklearn.datasets import load_boston
import sklearn.linear_model as sk
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
path = "/home/alexander/Downloads/opalcnn" 
model = keras.models.load_model(path,custom_objects={'F1Score': F1Score} )

In [27]:
def convert_to_int(chars):
    low_byte = int(chars[:2], 16)
    high_byte = int(chars[2:], 16)
    # Perform sign extension if the high byte is negative
    if high_byte & 0x80:
        high_byte -= 0x100
    # Combine the bytes and return the result
    return (high_byte << 8) | low_byte

In [28]:
import math
import statistics
def make_pred(FL):
    i=0
    sJ = [0]*300
    cirR = [0]*300
    cirI = [0]*300
    for j in range(0,300):     
        cirR[j] =  convert_to_int(FL[i:i+4])
        cirI[j] =  convert_to_int(FL[i+4:i+8])
        sJ[j]=j
        i = i+8 
    final = [0]*300
    for i in range(len(cirI)):
        cirI[i] = cirI[i] * cirI[i]
        cirR[i] = cirR[i] * cirR[i]
        final[i] = math.sqrt(cirR[i]+cirI[i])
    
    # Calculate the mean of the list
    #print(final)
    mean = np.mean(final)
    #print("Mean:", mean)

    # Calculate the standard deviation of the list
    std_dev = np.std(final)

    final = (final - mean)/std_dev        
    zero = np.array(final)
    zero = zero.reshape(-1,300)
    #print(zero)
    pred = model.predict(zero)
    print(pred)
    if pred>0.5:
        print('nlos')
    else:
        print('los')

In [ ]:
#all good with this one
import pylink
import time

# Create a PyLink instance for the SEGGER J-Link debug probe
jlink = pylink.JLink()

# Connect to the target device
jlink.open()
jlink.connect("NRF52832_XXAA")

# Start the RTT viewer on the host computer
jlink.rtt_start()
i=0
flag=0
msg=''
# Set up a polling loop to read data from the RTT buffer
while 1:
    # Read any available data from the RTT buffer
    data = jlink.rtt_read(0,1024)

    
    # Convert the list to a bytes object
    data_bytes = bytes(data)
    
    
    if data_bytes !=b'':
        d = data_bytes.decode('utf-8')
        
        if d.endswith('\n') and msg!='':
            d = msg+d
            #print(d)
            flag=1
            msg=''
        elif d.endswith('\n') and msg=='':
            #print(d)
            flag=1        
              
        elif msg=='':
            msg =d
            #print(msg)
            #print('msg')
            flag=0
        elif msg!='':
            msg=msg+d
            flag=0
            
    if flag==1:
        lines = d.split('\n')
        flag=0
        for line in lines:
            if 'FAIL' in line:
                print('We have an error')
            elif 'SUCCESS' and 'RNG' in line:
                # Extract the relevant numbers from the line
                parts = line.split()
                bias_index = parts.index('bias')
                try_index = parts.index('RNG')
                success_num = parts[bias_index+1]
                try_num = parts[try_index+1]
                print(f'Try: {try_num} Success {success_num}')
            elif 'CIR' in line:
                parts = line.split()
                n_index = parts.index('300')
                cir_in = parts[n_index+1]
                make_pred(cir_in)
                #print(cir_in)
                #print(len(cir_in))
        
        
       
    time.sleep(0.01)


We have an error
We have an error
We have an error
Try: 2 Success 177
[[0.27882147]]
los
We have an error
We have an error
Try: 3 Success 176
[[0.22160387]]
los
We have an error
We have an error
Try: 4 Success 174
[[0.34074]]
los
We have an error
We have an error
Try: 5 Success 174
[[0.1685394]]
los
We have an error
We have an error
Try: 6 Success 173
[[0.15602818]]
los
We have an error
We have an error
Try: 7 Success 176
[[0.17210686]]
los
We have an error
We have an error
Try: 8 Success 172
[[0.1473014]]
los
We have an error
We have an error
Try: 9 Success 177
[[0.3723951]]
los
We have an error
We have an error
Try: 10 Success 173
[[0.541133]]
nlos
We have an error
We have an error
Try: 11 Success 196
[[0.59705955]]
nlos
We have an error
We have an error
Try: 12 Success 195
[[0.62522644]]
nlos
We have an error
We have an error
Try: 13 Success 191
[[0.5373165]]
nlos
We have an error
We have an error
Try: 14 Success 187
[[0.4274894]]
los
We have an error
We have an error
Try: 15 Succes